In [1]:
# -*- coding: utf-8 -*-
import sys
import os
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import xlrd
import openpyxl

sys.path.append("..")
from config import *
from lib import *

In [2]:
targe_file = merge_result_output_filename
ldf = pd.read_excel(targe_file, index_col=[0])

patch_file = os.path.join(STORAGE_DIR, 'Tyr_custom.xlsx')
cdf = pd.read_excel(patch_file, index_col=[0])

xdf = ldf.join(cdf[['Custom', 'Custom_female']]).fillna('')

In [3]:
l_key = '[ali]DefaultText'
lf_key = '[ali]FemaleText'
r_key = '[3dm]DefaultText'
rf_key = '[3dm]FemaleText'
m_key = 'merged'
f_key = 'merged_female'

In [4]:
# 遍历行，按相应方式合并到 merged cell
def new_merge(xdf):
    merges = []
    merges_female = []

    for i,row in xdf.iterrows():
        new_cell = ''    
        if row['Custom'] != '':
            new_cell = row['Custom']
        elif not has_ch(row[l_key]) and has_ch(row[r_key]):
            new_cell = row[r_key]
        else:
            new_cell = row[l_key]
            
        merges.append(new_cell)

        new_f_cell = ''
        if row['Custom_female'] != '':
            new_f_cell = row['Custom_female']
        elif has_ch(row['[ali]FemaleText']):
            new_f_cell = row['[ali]FemaleText']
        elif not has_ch(row['[ali]FemaleText']) and has_ch(row['[3dm]FemaleText']):
            new_f_cell = row['[3dm]FemaleText']
        else:
            new_f_cell = row['FemaleText']
 
        merges_female.append(new_f_cell)

    xdf[m_key] = merges
    xdf[f_key] = merges_female
    return xdf
    
xdf = new_merge(xdf)

In [5]:
xdf.ix[41816]

Name                                           game\gui
ID                                                 4234
DefaultText         Unarmed [url=glossary:dps]DPS[/url]
FemaleText                                             
[ali]DefaultText         徒手[url=glossary:dps]伤害输出[/url]
[ali]FemaleText                                        
[3dm]DefaultText          徒手[url=glossary:dps]DPS[/url]
[3dm]FemaleText                                        
Custom                    徒手[url=glossary:dps]DPS[/url]
Custom_female                                          
merged                    徒手[url=glossary:dps]DPS[/url]
merged_female                                          
Name: game\gui_4234, dtype: object

In [6]:
# glossary 名词表替换
def gloss_patch(Series):
    gloss_file = os.path.join(STORAGE_DIR, 'Tyr_gloss.xlsx')
    gloss_list = pd.read_excel(gloss_file).fillna('')

    en_key = 'en'
    chs_key = 'chs'
    oth_key = 'oth'
    # code key
    reg = '\[url=glossary:(.*?)\]'
    gloss_list['code'] = gloss_list[en_key].str.extract(reg,expand=False)
    gloss_list['chs_code'] = gloss_list[chs_key].str.extract('\[url=.*?\](.*?)(?=\[/url\])',expand=False)
    gloss_list = gloss_list.fillna('')
    gloss_list.ix[57]

    for i, row in gloss_list.iterrows():  
        if row[chs_key] != '':
            re = '(\[url=glossary:'+row['code']+'\].*?\[/url\])'
            repl = '[url=glossary:{chs}]{chs}[/url]'.format(chs = row['chs_code'])
#             print('repl: ' + re + ' to ' + repl)
            Series = Series.str.replace(re, repl)
            if row[oth_key] != '':
                for k in row[oth_key].split(','):
                    re2 = '(\[url=glossary:'+ k +'\].*?\[/url\])'
#                     print('repl2: ' + re2 + ' to ' + repl)
                    Series = Series.str.replace(re2, repl)
    return Series

# xdf[m_key] = gloss_patch(xdf[m_key])
xdf[m_key] = gloss_patch(xdf[m_key]).apply(fix_text)
xdf[f_key] = gloss_patch(xdf[f_key]).apply(fix_text)

In [7]:
# name patch
def name_list(Series):
    n_file = os.path.join(STORAGE_DIR, 'Tyr_name.xlsx')
    n_list = pd.read_excel(n_file).fillna('')
    
    for i,row in n_list.iterrows():
        if row['chs'] != '' and row['oth'] != '':
            am = row['oth'].split(',')
            if row['en'] !='':
                am.append(row['en'])
                
            for k in am:
                k = k.replace('[','\\[').replace(']','\\]')
                Series = Series.str.replace(''.join(k), row['chs'])

    return Series

xdf[m_key] = name_list(xdf[m_key])
xdf[f_key] = name_list(xdf[f_key])
xdf[m_key].ix[47]

'“凭你和缚命官法庭的关系，可以阻止这次袭击，除非涅尔特或者艾休非常想对[url=glossary:图侬]图侬[/url]宣战…我觉得这未必不可能。”\n\n“为了大家好，你应该向[url=glossary:图侬]图侬[/url]报告，装装样子就够了。”\n\n“无论如何，你对[url=glossary:图侬]图侬[/url]的忠诚会从执政官手中保护我们，所以你最好尽量维持和缚命官法庭的关系。不然”'

In [8]:
# fix ->
def gui_patch(Series):
    patch_file = os.path.join(STORAGE_DIR, 'Tyr_guifix.xlsx')
    patch_list = pd.read_excel(patch_file, index_col=0).fillna('')
    
    new_s = Series.copy()
    for i, row in patch_list.iterrows():
        if row['Custom'] != '':
            new_s[i] = row['Custom']
    return new_s
    
            
# xdf[m_key] = gui_patch(xdf[m_key])
xdf[m_key] = gui_patch(xdf[m_key])
xdf[m_key].ix[41924]

'[{0}{1}({2}~{3})]'

In [10]:
# 未翻译的行
def not_ch(text):
    return not has_ch(text)
xdf[xdf[l_key].apply(not_ch)].drop_duplicates(l_key).to_excel(os.path.join(TEMP_DIR,'notrans.xlsx'))

In [11]:
# 不对称或缺失引号，fix_text 处理后，未对称的引号会使用全角符号，方便查找
# ＇ ＂
doub = xdf[xdf[m_key].str.contains('＂.*?')]
sign = xdf[xdf[m_key].str.contains('＇.*?')]
tign = pd.merge(doub, sign, how='outer')
tign[['Name','ID','DefaultText',l_key,m_key, f_key]].to_excel(os.path.join(TEMP_DIR,'quote.xlsx'))

In [12]:
# 后期依次修改，先期转换为英文标点使用
xdf[m_key] = xdf[m_key].str.replace('＂','"')
xdf[m_key] = xdf[m_key].str.replace('＇',"'")

In [13]:
xdf[m_key].ix["conversations\\00_stronghold\\00_cv_act_2_alliescompanion_introduction_37"]
xdf[m_key].ix["conversations\\00_stronghold\\00_cv_pillar_first_arrival_act1_end_anarchist_40"]

"“记下。我相信他宁愿蹲坐在荆棘地里也不会再去[url=glossary:vendrien'凡德里恩之井[/url]的。毕竟全是糟糕的回忆。不过如果是给某个无礼的缚命官带路参观他们住所的话，他可能会改变主意。”她对你做了个鬼脸。“把这当做警告。”"

In [16]:
# 生成处理后的文件，只包含merged cell
final_file = os.path.join(TEMP_DIR,'Tyr_final.xlsx')
xdf[[m_key,f_key]].to_excel(final_file, index_label='index')

# 参考用
# xdf.to_excel(os.path.join(TEMP_DIR,'Tyr_xdf.xlsx'))

In [14]:
# out
from v_output import *
# final_filename = os.path.join(TEMP_DIR, 'xdf.xlsx')
en_strtbl_path = os.path.join(ORIGIN_DIR, 'Tyranny/1.0/en')
output_strtbl_path = os.path.join(OUTPUT_DIR, 'Tyr/Vmod')
xmlfilename = os.path.join(output_strtbl_path, "language.xml")

# 使用处理后的文件
# final_file = os.path.join(TEMP_DIR,'Tyr_final.xlsx')
# final_df = pd.read_excel(final_file,index_col=0).fillna('')
# df_outstring(final_df, en_strtbl_path, output_strtbl_path, m_key,f_key)

# 使用df数据
# df_outstring(xdf, en_strtbl_path, output_strtbl_path, m_key,f_key)
# gen_xml(xmlfilename)